In [ ]:
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import rich.table
import planetary_computer
from pystac_client import Client

from IPython.display import Image

## 6/27 END OF DAY
- took a bit to figure out where to find my planetary computer account api key
    here: https://github.com/microsoft/PlanetaryComputer/discussions/77 
    - added my API key as an environment variable and restarted the conda env but now the kernel dies whenever I try to run a command? not sure what's up... 

Following instructions from the [S1 RTC Example notebook](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc#Example-Notebook)

In [ ]:
catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
catalog

Specify time range and bounding box of interest: 

In [41]:
time_range = '2015-01-01/2022-05-01'
bbox = [88.214935, 27.932767, 88.263184,  28.017289]

Search for items in the collection you're interested in (here, S1 RTC) matching those parameters. 
Looks like there are 100 scenes in the dataset that match the criteria I specified:

In [42]:
search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

100

Convert to geodataframe and explore:

In [43]:
df = gpd.GeoDataFrame.from_features(items.to_dict(), crs='epsg:4326')
df.head(1)

,geometry,datetime,platform,s1:shape,proj:bbox,proj:epsg,proj:shape,end_datetime,constellation,s1:resolution,...,sar:center_frequency,sar:resolution_range,s1:product_timeliness,sar:resolution_azimuth,sar:pixel_spacing_range,sar:observation_direction,sar:pixel_spacing_azimuth,sar:looks_equivalent_number,s1:instrument_configuration_ID,sat:platform_international_designator
0,"POLYGON ((89.80391 27.02294, 89.82898 27.27428...",2022-04-30T00:03:27.201903Z,SENTINEL-1A,"[27531, 21084]","[530180.0, 2980920.0, 798685.0, 3190410.0]",32645,"[27531, 21084]",2022-04-30 00:03:39.701264+00:00,Sentinel-1,high,...,5.405,20,Fast-24h,22,10,right,10,4.4,7,2014-016A


In [44]:
df.explore()

In [45]:
item0 = items[0]

Taking a look at a false color composite from a combination of VV, VH bands from the first item in the `items` list: 

In [46]:
Image(url=item0.assets['rendered_preview'].href)

## Inspect STAC metadata


In [47]:
table = rich.table.Table('key','value')
for k, v in sorted(item0.properties.items()):
    table.add_row(k, str(v))
    
table

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ key                                   ┃ value                                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ constellation                         │ Sentinel-1                                        │
│ datetime                              │ 2022-04-30T00:03:27.201903Z                       │
│ end_datetime                          │ 2022-04-30 00:03:39.701264+00:00                  │
│ platform                              │ SENTINEL-1A                                       │
│ proj:bbox                             │ [530180.0, 2980920.0, 798685.0, 3190410.0]        │
│ proj:epsg                             │ 32645                                             │
│ proj:shape                            │ [27531, 21084]                                    │
│ proj:transform                        │ [10.0, 0.0, 524470.0, 0.0, -10.0, 3191680.0, 0.0, │
│                                       │ 0.0, 1.0]                                         │
│ s1:datatake_id                        │ 336414                                            │
│ s1:instrument_configuration_ID        │ 7                                                 │
│ s1:orbit_source                       │ RESORB                                            │
│ s1:processing_level                   │ 1                                                 │
│ s1:product_timeliness                 │ Fast-24h                                          │
│ s1:resolution                         │ high                                              │
│ s1:shape                              │ [27531, 21084]                                    │
│ s1:slice_number                       │ 12                                                │
│ s1:total_slices                       │ 17                                                │
│ sar:center_frequency                  │ 5.405                                             │
│ sar:frequency_band                    │ C                                                 │
│ sar:instrument_mode                   │ IW                                                │
│ sar:looks_azimuth                     │ 1                                                 │
│ sar:looks_equivalent_number           │ 4.4                                               │
│ sar:looks_range                       │ 5                                                 │
│ sar:observation_direction             │ right                                             │
│ sar:pixel_spacing_azimuth             │ 10                                                │
│ sar:pixel_spacing_range               │ 10                                                │
│ sar:polarizations                     │ ['VV', 'VH']                                      │
│ sar:product_type                      │ GRD                                               │
│ sar:resolution_azimuth                │ 22                                                │
│ sar:resolution_range                  │ 20                                                │
│ sat:absolute_orbit                    │ 42995                                             │
│ sat:orbit_state                       │ descending                                        │
│ sat:platform_international_designator │ 2014-016A                                         │
│ sat:relative_orbit                    │ 48                                                │
│ start_datetime                        │ 2022-04-30 00:03:14.702542+00:00                  │
└───────────────────────────────────────┴───────────────────────────────────────────────────┘

In [48]:
item0.properties["sar:polarizations"]

['VV', 'VH']

## Visualize assets

load `vv` data into xarray and plot results. we'll use dask here, see scaling with dask link in example notebook

In [50]:
from distributed import Client

client = Client(processes=False)
print(client.dashboard_link)

http://128.117.11.172:54087/status


/Users/emarshall/miniconda3/envs/s1_book/lib/python3.10/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54087 instead
  warnings.warn(


In [36]:
import stackstac

ds = stackstac.stack(
    planetary_computer.sign(items), bounds_latlon=bbox, epsg=32645, resolution = 100
)
ds

HTTPError: 403 Client Error: Forbidden for url: https://planetarycomputer.microsoft.com/api/sas/v1/token/sentinel1euwestrtc/sentinel1-grd-rtc

In [38]:
import pystac
import planetary_computer
import rioxarray

item_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/S1A_IW_GRDH_1SDV_20220627T001933_20220627T001958_043841_053BDB_rtc"

# Load the individual item metadata and sign the assets
item = pystac.Item.from_file(item_url)
signed_item = planetary_computer.sign(item)

# Open one of the data assets (other asset keys to use: 'vv')
asset_href = signed_item.assets["vh"].href
ds = rioxarray.open_rasterio(asset_href)
ds

HTTPError: 403 Client Error: Forbidden for url: https://planetarycomputer.microsoft.com/api/sas/v1/token/sentinel1euwestrtc/sentinel1-grd-rtc